In [1]:


import pandas as pd
import numpy as np
import os


import library as lib
from library import GlobalVars
from library import ExperimentConfig
import data_library as data_lib

import faiss_experiment as faiss_tool
import helpers.config_executor as executor


import experiments_configurations as ex_cfg
#
lib.init_random_seed(lib.RANDOM_SEED)

index_type="ivf_all"
faiss_experiments_output_file = GlobalVars.experiments_path + f"executed_faiss_experiments_index_{index_type}.csv"
all_faiss_experiments_file_path= GlobalVars.experiments_path + f"all_faiss_experiments_index_{index_type}.csv"
lib.combine_multiple_parameters_v3(all_faiss_experiments_file_path, ex_cfg.ivf_all)




In [2]:
"""
Execute ONE configuration as test calling individual process
"""
os.environ["LOKY_MAX_CPU_COUNT"] = "8"
debug_mode=True
cfg_records=pd.read_csv(all_faiss_experiments_file_path)
row = cfg_records.iloc[0].copy()
new_row = executor.launch_execute_configurations_as_process(faiss_tool, row, debug_mode=debug_mode)
print("results=", new_row)

stdout: configs json= {'segment_lenght': 10, 'segment_overlap': 'all', 'feature': 'pe-mfcc_40', 'vector_operation': 'mean', 'metric_type': 'cosine', 'vote_type': 'uniform', 'neighbors': 15, 'index_params': "{'params':'IVF1000,Flat', 'nprobe': 20}", 'normalize': 1, 'balanced_type': 1}
Param random seed: 0
index params= {'params':'IVF1000,Flat', 'nprobe': 20} <class 'str'>
index_specific_params= {'params': 'IVF1000,Flat', 'nprobe': 20} <class 'dict'>
train:/Volumes/Extreme SSD/MASTER-AI-2023/_Disertatie/bees-analyze-vectordb/app/../dataset/extended_segments_10_sec_overlap_5/__vectors_pe-mfcc_40_mean.cache
test:/Volumes/Extreme SSD/MASTER-AI-2023/_Disertatie/bees-analyze-vectordb/app/../dataset/extended_segments_10_sec_overlap_5/__vectors_pe-mfcc_40_mean.cache
use cache=True;file exists=False;path=/Volumes/Extreme SSD/MASTER-AI-2023/_Disertatie/bees-analyze-vectordb/app/../faiss_index/pe_mfcc_40_vectors_mean_len10_overlapallIVF1000,Flat_None_cosine.index.faiss
random seed was set to: 42
i

In [5]:
"""
Execute ONE configuration as test by library function call
"""
debug_mode=True
cfg_records=pd.read_csv(all_faiss_experiments_file_path)
row = cfg_records.iloc[0].copy()
print(row)
print("index params=", row['index_params'], type(row["index_params"]))
cfg = ExperimentConfig(row)
GlobalVars.set_segment_lenght_and_overlap(cfg._SEGMENT_LENGHT, cfg._SEGMENT_OVERLAP)


faiss_results = faiss_tool.execute_configuration(cfg)
print("results=", faiss_results)

segment_lenght                                           10
segment_overlap                                         all
feature                                          pe-mfcc_40
vector_operation                                       mean
metric_type                                          cosine
vote_type                                           uniform
neighbors                                                15
index_params        {'params':'IVF1000,Flat', 'nprobe': 20}
Name: 0, dtype: object
index params= {'params':'IVF1000,Flat', 'nprobe': 20} <class 'str'>
index_specific_params= {'params': 'IVF1000,Flat', 'nprobe': 20} <class 'dict'>
train:/Volumes/Extreme SSD/MASTER-AI-2023/_Disertatie/bees-analyze-vectordb/app/../dataset/extended_segments_10_sec_overlap_5/__vectors_pe-mfcc_40_mean.cache
test:/Volumes/Extreme SSD/MASTER-AI-2023/_Disertatie/bees-analyze-vectordb/app/../dataset/extended_segments_10_sec_overlap_5/__vectors_pe-mfcc_40_mean.cache
use cache=True;file exists=False;pa

In [ ]:
'''
Execute all configurations from all_faiss_experiments_file_path
'''
exec.debug_mode=False
do_normalize = True
debug_mode = False
repeat_no = 5
index_type = 'hnsw_efsearch_50'


all_faiss_experiments_file_path = 'experiments' + os.sep + f"all_faiss_experiments_index_{index_type}.csv"


lib.combine_multiple_parameters_v3(all_faiss_experiments_file_path, ex_cfg.hnsw_efsearch_50)
#cache_all_faiss_indexes(all_faiss_experiments_file_path)

for i in range(repeat_no):
    print(f"Exceute round {i}")
    faiss_experiments_output_file = 'experiments' + os.sep + f"executed_faiss_experiments_index_{index_type}_round{i}.csv"
    #exec.process_all_configs_by_threads(exec_tool=faiss_tool, input_file_path=all_faiss_experiments_file_path, output_file_path=faiss_experiments_output_file, num_threads = 4, chunk_size = 20)


""" 
all rounds = 3 min    
"""

AttributeError: 'builtin_function_or_method' object has no attribute 'debug_mode'